In [77]:
import numpy as np
import torch as pt
import pandas as pd
import json
import string

In [2]:
def jsonlToDataset(jsonl_path):
    dataset = []
    with open(jsonl_path, 'r') as json_file:
        json_list = list(json_file)
    for json_str in json_list:
        dataset.append(json.loads(json_str))
    return dataset

In [3]:
train_data_path = "D:/Deep_learning_laboratory/assignment_2/assignment_2_data/train.jsonl"
test_data_path = "D:/Deep_learning_laboratory/assignment_2/assignment_2_data/test.jsonl"
train_dataset = jsonlToDataset(train_data_path)
test_dataset = jsonlToDataset(test_data_path)

In [4]:
assert len(train_dataset) == 40398
assert len(test_dataset) == 1267

In [5]:
train_dataset[0]['sentence']

"Ian volunteered to eat Dennis's menudo after already having a bowl because _ despised eating intestine."

In [23]:
#idea: use RNN (plain). we need data = (sentence, option1, option2), label = (answer)

In [11]:
#https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html#sphx-glr-auto-examples-tutorials-run-word2vec-py

from gensim.models import KeyedVectors
wv = KeyedVectors.load("D:/Deep_learning_laboratory/assignment_2/word2vec-google-news-300", mmap='r')

In [12]:
# CODE TO DOWNLOAD WORD2VEC MODEL
#import gensim.downloader as api
#wv = api.load('word2vec-google-news-300')
#wv.save("word2vec-google-news-300")

In [13]:
pairs = [
    ('car', 'minivan'),   
    ('car', 'bicycle'),   
    ('car', 'airplane'),  
    ('car', 'cereal'),    
    ('capitalism', 'communism'), # 60% WTF
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, wv.similarity(w1, w2)))

'car'	'minivan'	0.69
'car'	'bicycle'	0.54
'car'	'airplane'	0.42
'car'	'cereal'	0.14
'capitalism'	'communism'	0.60


### Create a sample Dataset and remove punctuation

In [114]:
sample_dataset = []
for i in range(80):
    sample_dataset.append(train_dataset[i]['sentence'].split(maxsplit=-1))

In [115]:
def remove_punctuation(dataset):
    punctuation_to_remove = string.punctuation
    punctuation_to_remove = punctuation_to_remove.replace("_","")
    for i in range(len(dataset)):
        dataset[i] = dataset[i].translate(str.maketrans('', '', punctuation_to_remove))

In [116]:
remove_punctuation(sample_dataset[0])
sample_dataset[0]

['Ian',
 'volunteered',
 'to',
 'eat',
 'Denniss',
 'menudo',
 'after',
 'already',
 'having',
 'a',
 'bowl',
 'because',
 '_',
 'despised',
 'eating',
 'intestine']

### Stemming or Not?
Do we need to stem words before feeding them to the Word2Vec model? Better not, the result below shows us that stemming.

In [117]:
from nltk.stem import PorterStemmer
sample_dataset_stemmed = []
def preprocess_words(words):
    processed_words = []
    stemmer=PorterStemmer()
    for word in words:
        processed_words.append(stemmer.stem(word))
    return processed_words
for i in range(80):
    sample_dataset_stemmed.append(preprocess_words(sample_dataset[i]))

In [118]:
def number_of_words_of_phrase_not_in_Word2Vec(phrase):
    numberOfWordsNotInWord2Vec = 0
    for s in stri:
        try:
            result.append(wv[s])
        except KeyError:
            numberOfWordsNotInWord2Vec = numberOfWordsNotInWord2Vec + 1
    return numberOfWordsNotInWord2Vec

numberOfWordsNotInWord2Vec = 0
numberOfStemmedWordsNotInWord2Vec = 0

for i in range(80):
    numberOfWordsNotInWord2Vec += number_of_words_of_phrase_not_in_Word2Vec(sample_dataset[i])
    numberOfStemmedWordsNotInWord2Vec += number_of_words_of_phrase_not_in_Word2Vec(sample_dataset_stemmed[i]) 

print("There are " + str(numberOfWordsNotInWord2Vec) + " missing words, and " + str(numberOfStemmedWordsNotInWord2Vec) + " missing stemmed words in the Word2Vec model")

#TODO
# word "to" is missing. What to do with missing words?
# Dennis is reconognized, while Dennis's is not. We need to preprocess the data.

There are 320 missing words, and 320 missing stemmed words in the Word2Vec model


### Since there are the same number of stemmed words missing in Word2Vec as the number of unstemmed words, stemming won't be used
## How to handle missing words? (Words that are not present in Word2Vec model)

In [68]:
sample_phrase = sample_dataset[i]
sample_phrase

['Kevin',
 'is',
 'far',
 'more',
 'physically',
 'versatile',
 'than',
 'Nick,',
 'because',
 '_',
 'is',
 'a',
 'seasoned',
 'athlete',
 'in',
 'training.']

In [137]:
shape = wv[sample_phrase[0],].shape
a = pt.FloatTensor(shape[0], shape[1])
a.shape

torch.Size([1, 300])

In [145]:
sample_phrase_embeddings = []
for word in sample_phrase:
    try:
        sample_phrase_embeddings.append(pt.from_numpy(wv[word]))
    except KeyError:
        sample_phrase_embeddings.append(pt.FloatTensor(shape[0], shape[1])) #random distribution

### Now we can build a Generator class, to apply the previous steps to the whole dataset